In [2]:
!pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=63ca40623839f7f31f080bdc35382cbf8ce8c8596507482c4eb306d4a86b64dd
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta


In [3]:
import pandas as pd
from ta.trend import SMAIndicator, EMAIndicator, MACD
from ta.momentum import RSIIndicator
from ta.volatility import BollingerBands, AverageTrueRange

# Load your cleaned long-format data if needed
long_df = pd.read_csv("/content/drive/MyDrive/algo_trading_project/data/Cleaned_Nifty50_Data.csv", parse_dates=["Date"])

# Ensure correct format
long_df["Date"] = pd.to_datetime(long_df["Date"])
feature_data = []

# Loop through each ticker group
for ticker, group in long_df.groupby("Ticker"):
    df_t = group.sort_values("Date").copy()

    # Trend indicators
    df_t["SMA_20"] = SMAIndicator(close=df_t["Close"], window=20).sma_indicator()
    df_t["EMA_20"] = EMAIndicator(close=df_t["Close"], window=20).ema_indicator()

    # MACD
    macd = MACD(close=df_t["Close"])
    df_t["MACD"] = macd.macd()
    df_t["MACD_Signal"] = macd.macd_signal()

    # RSI
    df_t["RSI"] = RSIIndicator(close=df_t["Close"], window=14).rsi()

    # Bollinger Bands
    bb = BollingerBands(close=df_t["Close"], window=20, window_dev=2)
    df_t["BB_High"] = bb.bollinger_hband()
    df_t["BB_Low"] = bb.bollinger_lband()

    # ATR
    atr = AverageTrueRange(high=df_t["High"], low=df_t["Low"], close=df_t["Close"])
    df_t["ATR"] = atr.average_true_range()

    # Collect enhanced data
    feature_data.append(df_t)

# Combine all tickers
feature_df = pd.concat(feature_data, ignore_index=True)

# Drop rows with missing indicator values
feature_df.dropna(inplace=True)

# Optional: Save to file
feature_df.to_csv("nifty50_features.csv", index=False)
print("✅ Feature-rich dataset saved to nifty50_features.csv")


✅ Feature-rich dataset saved to nifty50_features.csv


In [5]:
!cp /content/nifty50_features.csv /content/drive/MyDrive/algo_trading_project/data/